In [34]:
from elasticsearch import Elasticsearch
import json
from jsmin import jsmin
import sys
import StringIO

In [2]:
%install_ext https://raw.githubusercontent.com/szeitlin/watermark/master/watermark.py

Installed watermark.py. To use it, type:
  %load_ext watermark


In [3]:
%load_ext watermark

In [9]:
%watermark -a "Samantha Zeitlin" -d -u -p elasticsearch,minify,python

Samantha Zeitlin 
Last updated: 04/09/2015 

elasticsearch 1.4.0
minify 0.1.4
python 2.7.9


In [36]:
!cd elk_test
!ls elk_test

2013_2014_comparison_prep.ipynb         bike1year_viz2.ipynb                    compare_2013_2014.ipynb                 node_modules                            sorting_morning_afternoon.ipynb
26feb_2015_timeseries.ipynb             calories_pretty-2nd.ipynb               generate_period_list.ipynb              sorting_city_vs_suburb_trips.ipynb      testing.ipynb
9March_2015_ts_analyses.ipynb           calories_pretty.ipynb                   import-one-year.ipynb                   sorting_morning_afternoon-2013_v2.ipynb
bike-1year-viz.ipynb                    city_vs_suburb_sandbox.ipynb            import-second-year.ipynb                sorting_morning_afternoon-2014.ipynb


In [37]:
#this could be made more general later, should decide how many nbs to put per index(?), whether to specify mapping, id numbers or let ES do that
prefix = '{"index": {"_index": "test", "_type": "type1"}}\n'

In [38]:
#try doing two files together (if this works, put target files in folder and iterate through that)
with open("elk_test/import-one-year.ipynb", 'r') as test:
    minified = jsmin(test.read())
with open("elk_test/2013_2014_comparison_prep.ipynb", 'r') as test2:
    mini2 = jsmin(test2.read())
    with open('pyminified_testfile.json', 'wb') as pyminified:
        pyminified.write(prefix)
        pyminified.write(minified)
        pyminified.write(prefix)
        pyminified.write(mini2)

In [39]:
es = Elasticsearch()
es.indices.create(index='test-index')

{u'acknowledged': True}

In [40]:
es.indices.delete(index='test-index')

{u'acknowledged': True}

In [21]:
%%bash
curl -s -XPOST localhost:9200/_bulk --data-binary @test2.json


{"took":251,"errors":false,"items":[{"index":{"_index":"test","_type":"type1","_id":"2","_version":1,"status":201}}]}

In [ ]:
#need to decide whether to do this with requests or try the elasticsearch python wrapper or what. 
#looks like kibana barely works at all with only 1 document, suddenly with 2 it starts showing more functionality. 